In [1]:
import scipy
import os
import bcolz
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np

In [2]:

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()


def load_array(fname):
    return bcolz.open(fname)[:]


In [204]:
cloud_probs = pd.read_csv("data/test_preds_cloudy_model_10xtta.csv")

In [30]:
preds = pd.read_csv("data/bestof207.csv")
preds = preds.set_index(preds.image_name)
preds = preds.reindex(df_test.image_name)

In [59]:
# preds.tail()

In [60]:
filename = 'data/qin_best_197_small.hdf'


val = pd.read_hdf(filename,key="ltest")
train = pd.read_hdf(filename,key="train")
test = pd.read_hdf(filename,key="test")

In [68]:
test_probs = pd.DataFrame(test.values, columns = ["ID"]+labels.values())
test_probs.head(1)

,ID,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,test_0,0.00538746,1.33879e-06,0.000244487,0.0415807,0.000817549,0.997557,1.75734e-05,2.86734e-06,0.00223057,0.00164726,0.00247068,0.000588358,0.999969,0.00117396,0.00538777,1.4562e-05,0.00163985


In [72]:
test_probs = test_probs.set_index(test_probs.ID)
test_probs = test_probs.reindex(df_test.image_name)

In [8]:
y_train = []

df_train = pd.read_csv('data/train_v2.csv')
df_test = pd.read_csv('data/sample_submission_v2.csv')



In [4]:
label_map = {'slash_burn': 15, 'selective_logging': 14, 
             'cultivation': 8, 'clear': 5, 'habitation': 9,
             'conventional_mine': 7, 'cloudy': 6, 'primary': 12, 
             'water': 16, 'haze': 10, 'artisinal_mine': 1, 
             'partly_cloudy': 11, 'blooming': 3, 'bare_ground': 2, 
             'blow_down': 4, 'agriculture': 0, 'road': 13}

In [6]:
labels = {v: k for k, v in label_map.iteritems()}
labels.values()

['agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water']

In [13]:
for f, tags in tqdm_notebook(df_train.values, miniters=1000):
    targets = np.zeros(17)
    
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
        

    y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)


In [56]:
y_train[:,label_map["cloudy"]].sum()

4178

In [54]:
y_train[np.where(y_train[:,label_map["cloudy"]]==1)].sum()

4178

In [52]:
train_df_01 = pd.DataFrame(y_train, columns = labels.values())
train_df_01.head(1)

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0


complete mutual exclusion of cloudy with other tags

In [36]:
y_test = []


for f, tags in tqdm_notebook(preds.values, miniters=1000):
    targets = np.zeros(17)
    
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
        

    y_test.append(targets)
    
y_test = np.array(y_test, np.uint8)

In [268]:
y_test_fixed

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [274]:
y_test[np.where(y_test[:,label_map["cloudy"]]==1)].sum(axis=0)

array([ 979,   31,   24,    2,    4, 2166, 5249,    5,  221,  206,  993,
        990, 3374,  541,   14,    2, 1074], dtype=uint32)

In [147]:
preds_df_01 = pd.DataFrame(y_test, columns = labels.values())
preds_df_01.head(1)

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [116]:
(test_probs.cloudy.values < 0.05).sum()

55847

> 0.98   > 87
< 0.02 1136


cloud_probs
99 (730, 2)
< 0.01 : (1048, 2)


In [394]:
preds.iloc[np.where((y_test[:,label_map["cloudy"]]==1) & (y_test.sum(axis=1)<13) & (y_test.sum(axis=1)>1) & 
                    (y_test[:,label_map["water"]]==1) &
          (cloud_probs.cloudy.values >0.99))]

,image_name,tags
image_name,,
test_2079,test_2079,clear cloudy water
test_4269,test_4269,cloudy partly_cloudy primary water
test_4425,test_4425,clear cloudy water
test_6247,test_6247,clear cloudy primary water
test_8386,test_8386,cloudy haze primary water
test_9226,test_9226,clear cloudy water
test_17310,test_17310,cloudy haze primary water
test_17887,test_17887,clear cloudy water
test_20729,test_20729,cloudy partly_cloudy primary water


In [314]:
label_map["agriculture"], label_map["water"], label_map["clear"],label_map["cloudy"]

(0, 16, 5, 6)

In [380]:
y_test_fixed = y_test.copy()

#### remove cloud for those with low prob

In [381]:
remove_cloud_labels =  y_test[np.where((y_test[:,label_map["cloudy"]]==1) & (y_test.sum(axis=1)>1) &
          (cloud_probs.cloudy.values < 0.01 ))]

In [382]:
remove_cloud_labels[:,label_map["cloudy"]] = 0 

In [383]:
y_test_fixed[np.where((y_test[:,label_map["cloudy"]]==1) & (y_test.sum(axis=1)>1) &
          (cloud_probs.cloudy.values < 0.01 ))] = remove_cloud_labels

#### remove other labels with high cloud prob

In [384]:
remove_noncloud_labels =  y_test[np.where((y_test[:,label_map["cloudy"]]==1) & (y_test.sum(axis=1)>1) &
          (cloud_probs.cloudy.values > 0.99  ))]

In [385]:
remove_noncloud_labels*np.array([0]+[1]*3 + [0,1] + [1]*10+[0])

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [386]:
# remove_noncloud_labels = np.array([0]*5 + [1] + [0]*11)

In [387]:
label_map["clear"]

5

In [388]:
np.array([0]+[1]*4 + [0,1] + [1]*9+[0])

array([0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [389]:
#remove these only: label_map["agriculture"], label_map["water"], label_map["clear"],label_map["cloudy"]
y_test_fixed[np.where((y_test[:,label_map["cloudy"]]==1) & (y_test.sum(axis=1)>1) &
          (cloud_probs.cloudy.values  > 0.99 ))] = remove_noncloud_labels*np.array([0]+[1]*4 + [0,1] + [1]*9+[0])


# remove all other labels # np.array([0]*5 + [1] + [0]*11)

#### put in df and submit

In [390]:
preds_df_fixed_01 = pd.DataFrame(y_test_fixed, columns = labels.values())
preds_df_fixed_01.head(1)

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [391]:
label_preds = []
for i in tqdm_notebook(range(preds_df_fixed_01.shape[0]), miniters=1000):
    a = preds_df_fixed_01.ix[[i]]
    a = a.apply(lambda x: x > 0, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    label_preds.append(' '.join(list(a.index)))

C:\Users\Armin\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  app.launch_new_instance()


In [392]:
df_test["tags"] = label_preds
df_test.head()

,image_name,tags
0,test_0,clear primary
1,test_1,clear primary
2,test_2,partly_cloudy primary
3,test_3,agriculture clear cultivation primary
4,test_4,cloudy partly_cloudy primary


In [393]:
submission_file = 'subm/clean_cloudy_on_bestofa_low_and_selective_high_thres.csv'
df_test.to_csv(submission_file, index=False)